In [1]:
import os
import math
from os import path

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.applications.resnet import ResNet50
from sklearn.model_selection import train_test_split
from PIL import Image

In [2]:
tf.test.is_gpu_available()

True

In [3]:
gitw_path = path.join("data")
classes = [d for d in os.listdir(gitw_path) if path.isdir(path.join(gitw_path, d))]
classes

['Background',
 'Bowl',
 'CanOfCocaCola',
 'FryingPan',
 'Glass',
 'Jam',
 'Lid',
 'MilkBottle',
 'Mug',
 'OilBottle',
 'Plate',
 'Rice',
 'SaucePan',
 'Sponge',
 'Sugar',
 'VinegarBottle',
 'WashLiquid']

In [4]:
directories = []
files = {
    "file": [],
    "label": [],
    "frame": []
}

for c_name in classes:
    for img in os.listdir(path.join(gitw_path, c_name)):
        if path.isfile(path.join(gitw_path, c_name, img)):
            files["file"].append(path.join(c_name, img))
            files["label"].append(classes.index(c_name))
            files["frame"].append(int(img.split("_")[1]))

In [5]:
files = pd.DataFrame(files).sort_values(by=["label", "frame"]).reset_index(drop=True)
files.head()

,file,label,frame
0,Background\Frame_0320_bg.png,0,320
1,Background\Frame_0360_bg.png,0,360
2,Background\Frame_0400_bg.png,0,400
3,Background\Frame_0440_bg.png,0,440
4,Background\Frame_0480_bg.png,0,480


In [6]:
x_train, x_test = train_test_split(files.index, test_size=0.15)
len(x_train), len(x_test)

(4451, 786)

In [7]:
train_df, test_df = files.loc[x_train], files.loc[x_test]
train_df.to_csv(path.join(gitw_path, "train_dataset.csv"), index=False)
test_df.to_csv(path.join(gitw_path, "test_dataset.csv"), index=False)

In [8]:
class GITWDataset(keras.utils.Sequence):
    
    def __init__(self, data_path, df, classes, batch_size=32):
        self.X = [path.join(data_path, file) for file in df["file"].values]
        self.Y = keras.utils.to_categorical(df["label"].values)
        self.batch_size = batch_size
        self.path = data_path
        self.df = df
        self.classes = classes
        
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)
    
    def __getitem__(self, idx):
        files_x = self.X[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_y = self.Y[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_x = []
        for file in files_x:
            frame = np.array(Image.open(os.path.join(file)).convert("RGB").resize((224,224), Image.BICUBIC).getdata())
            batch_x.append(frame.reshape(224, 224, 3))
        return np.stack(batch_x), batch_y

In [9]:
trainset = GITWDataset(gitw_path, train_df, classes, batch_size=8)
testset = GITWDataset(gitw_path, test_df, classes, batch_size=8)

In [10]:
resnet_module = ResNet50(include_top=False, input_shape=(224, 224, 3), )
resnet = models.Sequential([
    resnet_module,
    layers.Flatten(),
    layers.Dense(len(classes), activation="softmax")
])

In [11]:
resnet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 17)                1706001   
Total params: 25,293,713
Trainable params: 25,240,593
Non-trainable params: 53,120
_________________________________________________________________


In [12]:
resnet.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
resnet.fit_generator(trainset, 
                    epochs=5,
                    validation_data=testset)

Epoch 1/5
  1/557 [..............................] - ETA: 53:50 - loss: 4.2258 - accuracy: 0.2500

ResourceExhaustedError: OOM when allocating tensor with shape[8,1024,14,14] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:FusedBatchNormV3]